In [6]:
!pip install stanza
!pip install nltk Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.9 MB/s eta 0:00:00


In [8]:
!pip install transformers datasets torch --quiet
!pip install huggingface_hub --quiet


In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import os

In [14]:
# =============================
# 0️⃣ Install & Import Library
# =============================

# Install package yang diperlukan
!pip install transformers stanza Sastrawi

import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

from collections import Counter
import torch
import torch.nn.functional as F

# NLP Library
import stanza
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from transformers import AutoModelForSequenceClassification

# =============================
# 1️⃣ Load Preprocessed CSV
# =============================
# Upload file ke Colab terlebih dahulu
from google.colab import files
uploaded = files.upload()

# Load dataset
df = pd.read_csv("berita_preprocessed.csv")  # hasil preprocessing sebelumnya


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Saving berita_preprocessed.csv to berita_preprocessed (1).csv


In [15]:

# =============================
# 2️⃣ POS Tagging (Stanza)
# =============================
# Download model stanza untuk bahasa Indonesia
stanza.download('id')
nlp_pos = stanza.Pipeline('id', processors='tokenize,pos', use_gpu=True)  # gunakan GPU untuk percepatan

def pos_tag_text(text):
    doc = nlp_pos(str(text))
    return [(word.text, word.upos) for sent in doc.sentences for word in sent.words]

df['POS Tags'] = df['Raw Clean (POS/NER)'].apply(pos_tag_text)

# =============================
# 3️⃣ NER (IndoBERT PyTorch)
# =============================
ner_model_name = "cahya/bert-base-indonesian-ner"
tokenizer_ner = AutoTokenizer.from_pretrained(ner_model_name)
model_ner = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner_pipe = pipeline("ner", model=model_ner, tokenizer=tokenizer_ner, aggregation_strategy="simple")

def ner_text(text):
    try:
        return ner_pipe(str(text))
    except Exception as e:
        print(f"Error in NER: {e}")
        return []

df['NER'] = df['Raw Clean (POS/NER)'].apply(ner_text)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: id (Indonesian) ...
INFO:stanza:File exists: /root/stanza_resources/id/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: id (Indonesian):
| Processor | Package    |
--------------------------
| tokenize  | gsd        |
| mwt       | gsd        |
| pos       | gsd_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Done loading processors!
Some weights of the model checkpoint at cahya/bert-base-indonesian-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initial

Error in NER: The size of tensor a (760) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (598) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (527) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (673) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (581) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (846) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (622) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (631) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of tensor a (872) must match the size of tensor b (512) at non-singleton dimension 1
Error in NER: The size of te

In [ ]:
# =============================
# 4️⃣ Sentiment Analysis Klasik (Lexicon Sederhana)
# =============================
positive_words = set(['baik', 'bagus', 'senang', 'puas', 'mantap', 'sukses', 'hebat', 'positif'])
negative_words = set(['buruk', 'jelek', 'sedih', 'kecewa', 'gagal', 'masalah', 'negatif', 'rugi'])

def sentiment_classic(text):
    try:
        tokens = str(text).split()  # pakai After Stopwords
        pos_count = sum(1 for w in tokens if w in positive_words)
        neg_count = sum(1 for w in tokens if w in negative_words)
        if pos_count > neg_count:
            return "Positive"
        elif neg_count > pos_count:
            return "Negative"
        else:
            return "Neutral"
    except Exception as e:
        print(f"Error in classic sentiment: {e}")
        return "Neutral"

# Pastikan kolom 'Sentiment After Stopwords' ada, jika tidak gunakan kolom lain
if 'Sentiment After Stopwords' in df.columns:
    df['Sentiment Classic'] = df['Sentiment After Stopwords'].apply(sentiment_classic)
else:
    df['Sentiment Classic'] = df['Raw Clean (POS/NER)'].apply(sentiment_classic)

# =============================
# 5️⃣ Sentiment Analysis Transformer (IndoBERT-Sentiment)
# =============================
sent_model_name = "indolem/indobert-base-uncased"
tokenizer_sent = AutoTokenizer.from_pretrained(sent_model_name)
model_sent = AutoModelForSequenceClassification.from_pretrained(sent_model_name)

def sentiment_transformer(text):
    try:
        inputs = tokenizer_sent(str(text), return_tensors='pt', truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = model_sent(**inputs)
        probs = F.softmax(outputs.logits, dim=1)
        label = torch.argmax(probs, dim=1).item()
        mapping = {0:'Negative', 1:'Neutral', 2:'Positive'}
        return mapping[label]
    except Exception as e:
        print(f"Error in transformer sentiment: {e}")
        return "Neutral"

# Pastikan kolom 'Sentiment Clean (Transformer)' ada
if 'Sentiment Clean (Transformer)' in df.columns:
    df['Sentiment Transformer'] = df['Sentiment Clean (Transformer)'].apply(sentiment_transformer)
else:
    df['Sentiment Transformer'] = df['Raw Clean (POS/NER)'].apply(sentiment_transformer)


In [ ]:

# =============================
# 6️⃣ Tampilkan Hasil
# =============================
print("DataFrame setelah processing:")
print(df.head())
print(f"\nShape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

# Simpan hasil
df.to_csv("berita_with_sentiment_ner.csv", index=False)
print("\nFile berhasil disimpan sebagai 'berita_with_sentiment_ner.csv'")

In [16]:
# =============================
# 4️⃣ Sentiment Analysis Klasik (Lexicon)
# =============================

# Expanded sentiment lexicon berdasarkan analisis berita kapal ferry
positive_words = set([
    # Kata dasar positif
    'baik', 'bagus', 'senang', 'puas', 'mantap', 'sukses', 'hebat', 'positif',
    # Operasional normal & lancar
    'normal', 'lancar', 'beroperasi', 'aktif', 'siap', 'siagakan', 'tambah',
    'perkuat', 'tingkatkan', 'tingkat', 'perbaiki', 'perbaikan', 'jamin',
    # Peningkatan layanan
    'meningkat', 'membaik', 'tumbuh', 'naik', 'lebih', 'canggih', 'modern',
    'baru', 'digital', 'online', 'mudah', 'cepat', 'efisien', 'efisiensi',
    # Kebijakan positif
    'gratis', 'diskon', 'murah', 'terjangkau', 'satu', 'harga', 'standar',
    'aman', 'nyaman', 'selamat', 'keselamatan', 'aman', 'proteksi', 'jaminan',
    # Respons positif
    'dukung', 'dukungan', 'respons', 'positif', 'apresiasi', 'sambut', 'sambutan',
    'ramai', 'padat', 'penuh', 'banyak', 'tinggi', 'besar', 'optimal', 'maksimal',
    # Kesuksesan
    'berhasil', 'sukses', 'selesai', 'tuntas', 'final', 'akhir', 'selesai',
    'rampung', 'tercapai', 'target', 'capai', 'raih', 'prestasi', 'kinerja'
])

negative_words = set([
    # Kata dasar negatif
    'buruk', 'jelek', 'sedih', 'kecewa', 'gagal', 'masalah', 'negatif', 'rugi',
    # Kecelakaan & insiden
    'kebakaran', 'terbakar', 'api', 'bakar', 'kecelakaan', 'tabrak', 'tabrakan',
    'tenggelam', 'karam', 'kandas', 'rusak', 'mati', 'mesin', 'mogok', 'henti',
    'berhenti', 'stop', 'terhambat', 'hambat', 'ganggu', 'ganguan', 'kendala',
    # Korban & kerugian
    'tewas', 'meninggal', 'luka', 'luka-luka', 'korban', 'hilang', 'evakuasi',
    'selamatkan', 'selamat', 'darurat', 'kritis', 'rawat', 'rumah', 'sakit',
    # Masalah operasional
    'macet', 'antre', 'antrean', 'menumpuk', 'padat', 'penuh', 'terjebak',
    'terlambat', 'lambat', 'delay', 'tunda', 'penundaan', 'batal', 'pembatalan',
    'cancel', 'refund', 'ganti', 'rugi', 'kerugian', 'rugikan',
    # Cuaca buruk
    'ombak', 'tinggi', 'gelombang', 'besar', 'cuaca', 'buruk', 'ekstrem',
    'angin', 'kencang', 'badai', 'storm', 'hujan', 'deras', 'petir', 'guntur',
    # Keluhan
    'keluh', 'keluhan', 'protes', 'keberatan', 'tuntut', 'tuntutan', 'demo',
    'mogok', 'berhenti', 'boikot', 'tolak', 'penolakan', 'kritik', 'kritikan'
])

neutral_words = set([
    # Informasi netral
    'jadwal', 'harga', 'tiket', 'tarif', 'rute', 'lintasan', 'pelabuhan',
    'dermaga', 'kapal', 'feri', 'ferry', 'penumpang', 'kendaraan', 'mobil',
    'motor', 'bus', 'truk', 'operasional', 'layanan', 'pemesanan', 'reservasi',
    'online', 'aplikasi', 'website', 'system', 'sistem', 'digital', 'elektronik',
    'informasi', 'pengumuman', 'pemberitahuan', 'edaran', 'surat', 'keputusan',
    'peraturan', 'kebijakan', 'strategi', 'rencana', 'program', 'proyek'
])

In [17]:

def sentiment_classic(text):
    try:
        tokens = str(text).lower().split()

        # Hitung frekuensi kata sentiment
        pos_count = sum(1 for w in tokens if w in positive_words)
        neg_count = sum(1 for w in tokens if w in negative_words)
        neu_count = sum(1 for w in tokens if w in neutral_words)

        # Debug info (opsional)
        if pos_count > 0 or neg_count > 0:
            found_pos = [w for w in tokens if w in positive_words]
            found_neg = [w for w in tokens if w in negative_words]
            print(f"Text: {text[:100]}...")
            print(f"Positive words found: {found_pos}")
            print(f"Negative words found: {found_neg}")
            print(f"Counts - Pos: {pos_count}, Neg: {neg_count}, Neu: {neu_count}")
            print("---")

        # Decision logic dengan threshold
        total_sentiment_words = pos_count + neg_count

        if total_sentiment_words == 0:
            return "Neutral"

        # Jika ada kata sentiment yang dominan
        if pos_count > neg_count and pos_count >= 1:
            return "Positive"
        elif neg_count > pos_count and neg_count >= 1:
            return "Negative"
        else:
            return "Neutral"

    except Exception as e:
        print(f"Error in classic sentiment: {e}")
        return "Neutral"

# Fungsi dengan weighting untuk konteks khusus
def sentiment_classic_weighted(text):
    try:
        tokens = str(text).lower().split()

        # Weighting untuk kata-kata tertentu
        strong_positive = set(['sukses', 'hebat', 'mantap', 'aman', 'selamat', 'gratis', 'diskon'])
        strong_negative = set(['tewas', 'meninggal', 'kebakaran', 'tenggelam', 'kandas', 'terbakar'])

        pos_score = 0
        neg_score = 0

        for token in tokens:
            if token in strong_positive:
                pos_score += 2
            elif token in strong_negative:
                neg_score += 2
            elif token in positive_words:
                pos_score += 1
            elif token in negative_words:
                neg_score += 1

        # Decision logic dengan threshold
        if pos_score > neg_score and pos_score >= 1:
            return "Positive"
        elif neg_score > pos_score and neg_score >= 1:
            return "Negative"
        else:
            return "Neutral"

    except Exception as e:
        print(f"Error in weighted sentiment: {e}")
        return "Neutral"

# Apply sentiment analysis
print("Melakukan sentiment analysis klasik...")

# Coba kedua metode
df['Sentiment Classic Simple'] = df['Raw Clean (POS/NER)'].apply(sentiment_classic)
df['Sentiment Classic Weighted'] = df['Raw Clean (POS/NER)'].apply(sentiment_classic_weighted)

# Gunakan yang weighted sebagai hasil utama
df['Sentiment Classic'] = df['Sentiment Classic Weighted']

Melakukan sentiment analysis klasik...
Text: JAKARTA, KOMPAS.com Manifest penumpang dan kendaraan adalah dokumen vital yang menjadi dasar keselam...
Positive words found: ['keselamatan', 'perkuat', 'dukung', 'naik', 'siap', 'keselamatan', 'digital']
Negative words found: ['antrean']
Counts - Pos: 7, Neg: 1, Neu: 27
---
Text: BENGKULU, KOMPAS.com Kepala Desa Kaana, Alamudin, di Pulau Enggano, Bengkulu, menyebutkan listrik te...
Positive words found: ['penuh', 'lebih', 'berhasil', 'lancar', 'aman', 'positif', 'sukses', 'sukses', 'berhasil', 'penuh', 'positif', 'dukungan', 'lancar', 'normal']
Negative words found: ['penuh', 'penuh']
Counts - Pos: 14, Neg: 2, Neu: 36
---
Text: KMP Tunu Pratama Jaya tenggelam di Selat Bali, Rabu 2 7 2025 malam, setelah kirim sinyal darurat tak...
Positive words found: ['tinggi', 'besar']
Negative words found: ['tenggelam', 'darurat', 'buruk', 'ombak', 'tinggi', 'besar']
Counts - Pos: 2, Neg: 6, Neu: 0
---
Text: KOMPAS.com PT Angkutan Sungai Danau dan Penyeb

In [18]:
# 5️⃣ Sentiment Analysis Transformer (IndoBERT-Sentiment)
# =============================
print("Melakukan sentiment analysis transformer...")

# Gunakan model yang lebih spesifik untuk sentiment analysis bahasa Indonesia
sent_model_name = "indolem/indobert-base-uncased"
# Alternatif: "w11wo/indonesian-roberta-base-sentiment-classifier"

try:
    tokenizer_sent = AutoTokenizer.from_pretrained(sent_model_name)
    model_sent = AutoModelForSequenceClassification.from_pretrained(sent_model_name)

    def sentiment_transformer(text):
        try:
            inputs = tokenizer_sent(
                str(text),
                return_tensors='pt',
                truncation=True,
                padding=True,
                max_length=512
            )
            with torch.no_grad():
                outputs = model_sent(**inputs)
            probs = F.softmax(outputs.logits, dim=1)
            label = torch.argmax(probs, dim=1).item()

            # Mapping untuk model indolem/indobert-base-uncased
            mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}

            return mapping[label]
        except Exception as e:
            print(f"Error in transformer sentiment for text: {e}")
            return "Neutral"

    # Apply transformer sentiment
    df['Sentiment Transformer'] = df['Raw Clean (POS/NER)'].apply(sentiment_transformer)

except Exception as e:
    print(f"Error loading transformer model: {e}")
    print("Menggunakan fallback sentiment...")
    df['Sentiment Transformer'] = df['Sentiment Classic']


Melakukan sentiment analysis transformer...


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [19]:
print("\n" + "="*50)
print("HASIL SENTIMENT ANALYSIS")
print("="*50)

# Statistik sentiment classic
classic_counts = df['Sentiment Classic'].value_counts()
print(f"\nSentiment Classic Distribution:")
for sentiment, count in classic_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {sentiment}: {count} ({percentage:.1f}%)")

# Statistik sentiment transformer
transformer_counts = df['Sentiment Transformer'].value_counts()
print(f"\nSentiment Transformer Distribution:")
for sentiment, count in transformer_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  {sentiment}: {count} ({percentage:.1f}%)")

# Perbandingan hasil
print(f"\nPerbandingan Metode:")
agreement = (df['Sentiment Classic'] == df['Sentiment Transformer']).sum()
agreement_pct = (agreement / len(df)) * 100
print(f"  Kesepakatan antara Classic dan Transformer: {agreement}/{len(df)} ({agreement_pct:.1f}%)")

print(f"\nContoh Hasil Sentiment Analysis:")
print(df[['Raw Clean (POS/NER)', 'Sentiment Classic', 'Sentiment Transformer']].head(10))

# Simpan lexicon untuk referensi
lexicon_info = {
    'positive_words_count': len(positive_words),
    'negative_words_count': len(negative_words),
    'neutral_words_count': len(neutral_words),
    'total_lexicon_words': len(positive_words) + len(negative_words) + len(neutral_words)
}

print(f"\nLexicon Information:")
print(f"  Positive words: {lexicon_info['positive_words_count']}")
print(f"  Negative words: {lexicon_info['negative_words_count']}")
print(f"  Neutral words: {lexicon_info['neutral_words_count']}")
print(f"  Total lexicon: {lexicon_info['total_lexicon_words']} words")


HASIL SENTIMENT ANALYSIS

Sentiment Classic Distribution:
  Positive: 196 (68.8%)
  Negative: 73 (25.6%)
  Neutral: 16 (5.6%)

Sentiment Transformer Distribution:
  Neutral: 200 (70.2%)
  Negative: 85 (29.8%)

Perbandingan Metode:
  Kesepakatan antara Classic dan Transformer: 29/285 (10.2%)

Contoh Hasil Sentiment Analysis:
                                 Raw Clean (POS/NER) Sentiment Classic  \
0  JAKARTA, KOMPAS.com Manifest penumpang dan ken...          Positive   
1  BENGKULU, KOMPAS.com Kepala Desa Kaana, Alamud...          Positive   
2  KMP Tunu Pratama Jaya tenggelam di Selat Bali,...          Negative   
3  KOMPAS.com PT Angkutan Sungai Danau dan Penyeb...          Positive   
4  KOMPAS.com Kala melihat kapal berwarna hijau t...          Positive   
5  BAUBAU, KOMPAS.com Sebuah video yang memperlih...          Positive   
6  LAMPUNG, KOMPAS.com Kekisruhan sempat terjadi ...          Positive   
7  JAKARTA, KOMPAS.com PT ASDP Indonesia Ferry Pe...          Positive   
8  JAKA

In [20]:
# Simpan hasil ke CSV
df.to_csv('hasil_analisis_lengkap.csv', index=False, encoding='utf-8')
print("Data berhasil disimpan ke 'hasil_analisis_lengkap.csv'")

Data berhasil disimpan ke 'hasil_analisis_lengkap.csv'


In [21]:
df

,Judul Berita,Tanggal Berita,Tag,Link,Raw Clean (POS/NER),Sentiment Clean (Transformer),Sentiment Before Stopwords,Sentiment After Stopwords,Tokens Before Stopwords,Tokens After Stopwords,Sentence Tokens,Word Tokens,POS Tags,NER,Sentiment Classic Simple,Sentiment Classic Weighted,Sentiment Classic,Sentiment Transformer
0,ASDP dan Operator Ferry Swasta Perkuat Data Ma...,2025-08-12 00:00:00,"ASDP, kapal ferry, manifest penumpang, operato...",https://money.kompas.com/read/2025/08/12/15391...,"JAKARTA, KOMPAS.com Manifest penumpang dan ken...","jakarta, kompas.com manifest penumpang dan ken...","jakarta , kompas.com manifest penumpang dan ke...",jakarta kompas com manifest tumpang kendara d...,"['jakarta', ',', 'kompas.com', 'manifest', 'pe...","['jakarta', '', 'kompas com', 'manifest', 'tum...","['JAKARTA, KOMPAS.com Manifest penumpang dan k...","['JAKARTA', ',', 'KOMPAS.com', 'Manifest', 'pe...","[(JAKARTA, PROPN), (,, PUNCT), (KOMPAS.com, PR...","[{'entity_group': 'GPE', 'score': 0.95238477, ...",Positive,Positive,Positive,Negative
1,"Listrik di Pulau Enggano Sudah 24 Jam, Kapal F...",2025-07-08 00:00:00,"pulau enggano, Bio Solar, Pelabuhan Pulau Baai...",https://regional.kompas.com/read/2025/07/08/10...,"BENGKULU, KOMPAS.com Kepala Desa Kaana, Alamud...","bengkulu, kompas.com kepala desa kaana, alamud...","bengkulu , kompas.com kepala desa kaana , alam...",bengkulu kompas com kepala desa kaana alamud...,"['bengkulu', ',', 'kompas.com', 'kepala', 'des...","['bengkulu', '', 'kompas com', 'kepala', 'desa...","['BENGKULU, KOMPAS.com Kepala Desa Kaana, Alam...","['BENGKULU', ',', 'KOMPAS.com', 'Kepala', 'Des...","[(BENGKULU, PROPN), (,, PUNCT), (KOMPAS.com, P...",[],Positive,Positive,Positive,Neutral
2,"Ombak Hingga 2,5 Meter Hambat Pencarian KMP Tu...",2025-07-03 02:09:12+00:00,"Berita Terkini, Berita Hari Ini, Berita Harian...",https://video.kompas.com/watch/1858879/ombak-h...,"KMP Tunu Pratama Jaya tenggelam di Selat Bali,...","kmp tunu pratama jaya tenggelam di selat bali,...",kmp tunu pratama jaya tenggelam di selat bali ...,kmp tunu pratama jaya tenggelam selat bal rab...,"['kmp', 'tunu', 'pratama', 'jaya', 'tenggelam'...","['kmp', 'tunu', 'pratama', 'jaya', 'tenggelam'...",['KMP Tunu Pratama Jaya tenggelam di Selat Bal...,"['KMP', 'Tunu', 'Pratama', 'Jaya', 'tenggelam'...","[(KMP, PROPN), (Tunu, PROPN), (Pratama, PROPN)...","[{'entity_group': 'PRD', 'score': 0.4988904, '...",Negative,Negative,Negative,Negative
3,Rute Ferry Batam–Johor Direncanakan Aktif Tahu...,2025-05-23 00:00:00,"kapal ferry, batam johor bahru ferry schedule,...",https://money.kompas.com/read/2025/05/23/22053...,KOMPAS.com PT Angkutan Sungai Danau dan Penyeb...,kompas.com pt angkutan sungai danau dan penyeb...,kompas.com pt angkutan sungai danau dan penyeb...,kompas com pt angkut sungai danau seberang asd...,"['kompas.com', 'pt', 'angkutan', 'sungai', 'da...","['kompas com', 'pt', 'angkut', 'sungai', 'dana...",['KOMPAS.com PT Angkutan Sungai Danau dan Peny...,"['KOMPAS.com', 'PT', 'Angkutan', 'Sungai', 'Da...","[(KOMPAS.com, PROPN), (PT, PROPN), (Angkutan, ...","[{'entity_group': 'ORG', 'score': 0.8430694, '...",Positive,Positive,Positive,Negative
4,"Menyeberangi Danau Toba, Perjalanan Naik Kapal...",2025-05-16 00:00:00,"kapal dari ajibata ke tomok, kapal ferry pelab...",https://travel.kompas.com/read/2025/05/16/0707...,KOMPAS.com Kala melihat kapal berwarna hijau t...,kompas.com kala melihat kapal berwarna hijau t...,kompas.com kala melihat kapal berwarna hijau t...,kompas com kapal warna hijau sandar labuh kem...,"['kompas.com', 'kala', 'melihat', 'kapal', 'be...","['kompas com', 'kapal', 'warna', 'hijau', 'san...",['KOMPAS.com Kala melihat kapal berwarna hijau...,"['KOMPAS.com', 'Kala', 'melihat', 'kapal', 'be...","[(KOMPAS.com, PROPN), (Kala, PROPN), (melihat,...",[],Positive,Positive,Positive,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,ASDP Waspadai Cuaca Buruk Ganggu Penyeberangan...,2022-12-23 12:32:38,"b